# Utils

> Utils function used in the environment.

In [1]:
#| default_exp utils

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import os
import numpy as np
import pandas as pd
import toml

In [12]:
#| export
def read_toml(  config_file= "../test_data/Config.toml" # Path to the config file
):
    "Read setting file. The File containes the relative path to source and target."
    config= toml.load(config_file)
    return config
    

In [13]:
#| export
# Here an example on how to structure a config.toml file.
config  = read_toml()
config

{'source': {'path': '../test_data/source',
  'filename_mesh': 'humerus_sx.key',
  'filename_landmarks': 'landmarks_source.fcsv'},
 'target': {'path': '../test_data/target',
  'filename_landmarks': 'landmarks_target.csv'}}

In [14]:
#| export
def _merge_path(path1:str , path2: str)-> str:
    "Join one or more path components. Return the join path in str type."
    return os.path.join(path1,path2)

In [15]:
# #| export
_merge_path(config['source']['path'],config['source']['filename_mesh'])

'../test_data/source/humerus_sx.key'

In [16]:
#| export
def multiple_targets():
    "This function checks if the there are multiple target subject."
    config = read_toml()
    path= config['target']['path']
    filename = config['target']['filename_landmarks']
    list_target= [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
    if not list_target:
        assert os.path.isfile(_merge_path(path, filename)), "No file with Landmarks"
        return False
    else:
        return list_target


In [17]:
#| export
multiple_targets()

['100014', '100065', '100084', '100114']

In [18]:
#| export
def read_k_file(path_to_file: str # File to read from 
                 )-> pd.DataFrame: # Dataframe of shape [n_nodes, [ID,x,y,z]]
    "This function read *NODES from .k/.key file."
    id_list = []
    node_coords = []
    find_nodes = False
    
    #assert os.path.exists(path_to_file), "Wrong path. Path not existent. Check the name/extension of the file."
    
    with open(path_to_file, 'r') as fp:
        for _, line in enumerate(fp):
            if line.startswith('*NODE'):
                find_nodes = True
                continue
            if find_nodes and line.startswith('*'):
                break
            if find_nodes:
                node_id = int(line[:8]) 
                node_x = float(line[8:24])
                node_y = float(line[24:40])
                node_z = float(line[40:56])
                id_list.append(node_id)
                node_coords.append([node_x, node_y, node_z])   
                 
    assert node_coords, "Nodes have NOT been read."
    
    df = pd.concat([pd.DataFrame(id_list), pd.DataFrame(node_coords)],axis=1)
    df.columns = ['Label - node id','x','y','z']
    return df

In [20]:
# #| export
read_k_file(_merge_path(config['source']['path'],config['source']['filename_mesh']))

,Label - node id,x,y,z
0,3091310,-181.550415,145.430511,404.375488
1,3091330,-43.251625,196.187363,194.874634
2,3091332,-37.076435,219.493759,205.082413
3,3091333,-33.503811,176.722672,190.791733
4,3091334,-27.463694,215.396118,203.790085
...,...,...,...,...
7826,3182470,-47.164852,171.558502,192.754669
7827,3182472,-45.163231,170.620392,190.355927
7828,3183714,-35.255299,217.536606,208.970779
7829,3183720,-38.128960,216.646057,209.831924


In [21]:
#| export
def read_csv_file(path_to_file: str)-> pd.DataFrame:
    "This function read csv files in format id, x, y,z. to Pandas DataFrame"
    with open(path_to_file) as fp:
            df = pd.read_csv(fp, header=None, comment='#')
            df= df.iloc[:,:4]
            df.columns = ['Label - node id','x','y','z']
    return df

In [22]:
#| export
df = read_csv_file(_merge_path(config['source']['path'],config['source']['filename_landmarks']))
df

,Label - node id,x,y,z
0,1,-183.063339,134.525345,388.490692
1,2,-161.198669,143.211426,373.034149
2,3,-146.246490,150.397263,350.175262
3,4,-132.589066,156.544540,325.771271
4,5,-118.436974,161.047623,302.355225
5,6,-103.345276,166.406738,278.470612
6,7,-88.987976,173.114761,255.046127
7,8,-73.376801,179.190704,232.277298
8,9,-59.285698,180.420685,207.781021
9,10,-49.864941,174.740372,182.173218


In [23]:
#| export
def from_df_to_np(df: pd.DataFrame)->np.ndarray:
    "This function transform a pd.Dataframe of shape [ID, x,y,z] to numpy array of coordinates [x,y,z]"
    if len(df.columns) == 4:
        return df.iloc[:,1:4].values
    
    elif len(df.columns) == 3:
        return df.values
    else:
        raise Exception("Invalid number of column in the Dataframe. Expected either 3 or 4 columns. ")

In [24]:
# #| export
# from_df_to_np(df) #return np.array

In [25]:
#| export
def to_ls_dyna(number):
    return '{:16f}'.format(number)

In [26]:
#| export
to_ls_dyna(6.8)

'        6.800000'

In [38]:
#| hide
import nbdev; nbdev.nbdev_export()